In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/binance/binance-public-data.git

Cloning into 'binance-public-data'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 282 (delta 96), reused 49 (delta 47), pack-reused 154 (from 1)
Receiving objects: 100% (282/282), 6.51 MiB | 41.91 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [3]:
%cd /kaggle/working/binance-public-data/python
!pip install -r requirements.txt

/kaggle/working/binance-public-data/python


In [4]:
!python download-kline.py -t spot -s BTCUSDT -i 1m #比特币历史以来1分钟快照数据

Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-02.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-03.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-04.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-05.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-06.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-07.zip

File Download: /kaggle/working/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-08.zip
[##################################################]
File Download: /kaggle/working/binance

In [13]:
import os
import zipfile
import pandas as pd
import sqlite3
from tqdm import tqdm

table_name = "bitcoin_data_1m"

# 连接到SQLite数据库，如果不存在则创建
conn = sqlite3.connect('bitcoin_data.db')
cursor = conn.cursor()

# 创建表
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    Open_time TEXT,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Volume REAL,
    Close_time TEXT,
    Quote_asset_volume REAL,
    Number_of_trades INTEGER,
    Taker_buy_base_asset_volume REAL,
    Taker_buy_quote_asset_volume REAL,
    Ignore REAL
)
"""
cursor.execute(create_table_query)

# 定义数据所在目录路径
data_dir = "/kaggle/working/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/1m"

# 存储所有zip文件及其对应的最后6位数字
zip_files_with_dates = []

# 遍历目录下的zip文件
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.zip'):
            # 提取文件名中最后6位数字
            date_str = file[-10:-4].replace('-', '')
            zip_file_path = os.path.join(root, file)
            zip_files_with_dates.append((date_str, zip_file_path))

# 按最后6位数字从小到大排序
zip_files_with_dates.sort(key=lambda x: x[0])

# 手动指定列名
columns = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time',
           'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',
           'Taker_buy_quote_asset_volume', 'Ignore']

# 按排序后的顺序读取数据文件并存入sqlite
for _, zip_file_path in tqdm(zip_files_with_dates):
    csv_file_name = os.path.basename(zip_file_path).replace('.zip', '.csv')
    # 解压zip文件
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extract(csv_file_name, os.path.dirname(zip_file_path))
    csv_file_path = os.path.join(os.path.dirname(zip_file_path), csv_file_name)
    # 使用pandas读取csv文件
    df = pd.read_csv(csv_file_path,header=None, names=columns)
    # 将数据插入到SQLite表中
    df.to_sql(table_name, conn, if_exists='append', index=False)
    # 删除解压后的csv文件
    os.remove(csv_file_path)

# 提交事务并关闭连接
conn.commit()
conn.close()
    

100%|██████████| 92/92 [00:30<00:00,  2.99it/s]


In [14]:
!rm -rf /kaggle/working/binance-public-data/python/data

In [15]:
!python download-kline.py -t spot -s BTCUSDT -i 3m #比特币历史以来3分钟快照数据

Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/3m/BTCUSDT-3m-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/3m/BTCUSDT-3m-2017-02.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/3m/BTCUSDT-3m-2017-03.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/3m/BTCUSDT-3m-2017-04.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/3m/BTCUSDT-3m-2017-05.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/3m/BTCUSDT-3m-2017-06.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/3m/BTCUSDT-3m-2017-07.zip

File Download: /kaggle/working/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/3m/BTCUSDT-3m-2017-08.zip
[##################################################]
File Download: /kaggle/working/binance

0it [00:00, ?it/s]


In [16]:
table_name = "bitcoin_data_3m"
data_freq = '3m'

# 连接到SQLite数据库，如果不存在则创建
conn = sqlite3.connect('bitcoin_data.db')
cursor = conn.cursor()

# 创建表
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    Open_time TEXT,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Volume REAL,
    Close_time TEXT,
    Quote_asset_volume REAL,
    Number_of_trades INTEGER,
    Taker_buy_base_asset_volume REAL,
    Taker_buy_quote_asset_volume REAL,
    Ignore REAL
)
"""
cursor.execute(create_table_query)

# 定义数据所在目录路径
data_dir = "/kaggle/working/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/"+data_freq

# 存储所有zip文件及其对应的最后6位数字
zip_files_with_dates = []

# 遍历目录下的zip文件
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.zip'):
            # 提取文件名中最后6位数字
            date_str = file[-10:-4].replace('-', '')
            zip_file_path = os.path.join(root, file)
            zip_files_with_dates.append((date_str, zip_file_path))

# 按最后6位数字从小到大排序
zip_files_with_dates.sort(key=lambda x: x[0])

# 手动指定列名
columns = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time',
           'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',
           'Taker_buy_quote_asset_volume', 'Ignore']

# 按排序后的顺序读取数据文件并存入sqlite
for _, zip_file_path in tqdm(zip_files_with_dates):
    csv_file_name = os.path.basename(zip_file_path).replace('.zip', '.csv')
    # 解压zip文件
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extract(csv_file_name, os.path.dirname(zip_file_path))
    csv_file_path = os.path.join(os.path.dirname(zip_file_path), csv_file_name)
    # 使用pandas读取csv文件
    df = pd.read_csv(csv_file_path,header=None, names=columns)
    # 将数据插入到SQLite表中
    df.to_sql(table_name, conn, if_exists='append', index=False)
    # 删除解压后的csv文件
    os.remove(csv_file_path)

# 提交事务并关闭连接
conn.commit()
conn.close()
    

100%|██████████| 92/92 [00:11<00:00,  7.77it/s]


In [17]:
!rm -rf /kaggle/working/binance-public-data/python/data
!python download-kline.py -t spot -s BTCUSDT -i 5m #比特币历史以来5分钟快照数据
table_name = "bitcoin_data_5m"
data_freq = '5m'

# 连接到SQLite数据库，如果不存在则创建
conn = sqlite3.connect('bitcoin_data.db')
cursor = conn.cursor()

# 创建表
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    Open_time TEXT,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Volume REAL,
    Close_time TEXT,
    Quote_asset_volume REAL,
    Number_of_trades INTEGER,
    Taker_buy_base_asset_volume REAL,
    Taker_buy_quote_asset_volume REAL,
    Ignore REAL
)
"""
cursor.execute(create_table_query)

# 定义数据所在目录路径
data_dir = "/kaggle/working/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/"+data_freq

# 存储所有zip文件及其对应的最后6位数字
zip_files_with_dates = []

# 遍历目录下的zip文件
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.zip'):
            # 提取文件名中最后6位数字
            date_str = file[-10:-4].replace('-', '')
            zip_file_path = os.path.join(root, file)
            zip_files_with_dates.append((date_str, zip_file_path))

# 按最后6位数字从小到大排序
zip_files_with_dates.sort(key=lambda x: x[0])

# 手动指定列名
columns = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time',
           'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',
           'Taker_buy_quote_asset_volume', 'Ignore']

# 按排序后的顺序读取数据文件并存入sqlite
for _, zip_file_path in tqdm(zip_files_with_dates):
    csv_file_name = os.path.basename(zip_file_path).replace('.zip', '.csv')
    # 解压zip文件
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extract(csv_file_name, os.path.dirname(zip_file_path))
    csv_file_path = os.path.join(os.path.dirname(zip_file_path), csv_file_name)
    # 使用pandas读取csv文件
    df = pd.read_csv(csv_file_path,header=None, names=columns)
    # 将数据插入到SQLite表中
    df.to_sql(table_name, conn, if_exists='append', index=False)
    # 删除解压后的csv文件
    os.remove(csv_file_path)

# 提交事务并关闭连接
conn.commit()
conn.close()
    

Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/5m/BTCUSDT-5m-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/5m/BTCUSDT-5m-2017-02.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/5m/BTCUSDT-5m-2017-03.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/5m/BTCUSDT-5m-2017-04.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/5m/BTCUSDT-5m-2017-05.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/5m/BTCUSDT-5m-2017-06.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/5m/BTCUSDT-5m-2017-07.zip

File Download: /kaggle/working/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/5m/BTCUSDT-5m-2017-08.zip
[##################################################]
File Download: /kaggle/working/binance

100%|██████████| 92/92 [00:07<00:00, 12.40it/s]


In [18]:
!rm -rf /kaggle/working/binance-public-data/python/data
!python download-kline.py -t spot -s BTCUSDT -i 15m #比特币历史以来15分钟快照数据
table_name = "bitcoin_data_15m"
data_freq = '15m'

# 连接到SQLite数据库，如果不存在则创建
conn = sqlite3.connect('bitcoin_data.db')
cursor = conn.cursor()

# 创建表
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    Open_time TEXT,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Volume REAL,
    Close_time TEXT,
    Quote_asset_volume REAL,
    Number_of_trades INTEGER,
    Taker_buy_base_asset_volume REAL,
    Taker_buy_quote_asset_volume REAL,
    Ignore REAL
)
"""
cursor.execute(create_table_query)

# 定义数据所在目录路径
data_dir = "/kaggle/working/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/"+data_freq

# 存储所有zip文件及其对应的最后6位数字
zip_files_with_dates = []

# 遍历目录下的zip文件
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.zip'):
            # 提取文件名中最后6位数字
            date_str = file[-10:-4].replace('-', '')
            zip_file_path = os.path.join(root, file)
            zip_files_with_dates.append((date_str, zip_file_path))

# 按最后6位数字从小到大排序
zip_files_with_dates.sort(key=lambda x: x[0])

# 手动指定列名
columns = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time',
           'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',
           'Taker_buy_quote_asset_volume', 'Ignore']

# 按排序后的顺序读取数据文件并存入sqlite
for _, zip_file_path in tqdm(zip_files_with_dates):
    csv_file_name = os.path.basename(zip_file_path).replace('.zip', '.csv')
    # 解压zip文件
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extract(csv_file_name, os.path.dirname(zip_file_path))
    csv_file_path = os.path.join(os.path.dirname(zip_file_path), csv_file_name)
    # 使用pandas读取csv文件
    df = pd.read_csv(csv_file_path,header=None, names=columns)
    # 将数据插入到SQLite表中
    df.to_sql(table_name, conn, if_exists='append', index=False)
    # 删除解压后的csv文件
    os.remove(csv_file_path)

# 提交事务并关闭连接
conn.commit()
conn.close()
    

Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/15m/BTCUSDT-15m-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/15m/BTCUSDT-15m-2017-02.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/15m/BTCUSDT-15m-2017-03.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/15m/BTCUSDT-15m-2017-04.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/15m/BTCUSDT-15m-2017-05.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/15m/BTCUSDT-15m-2017-06.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/15m/BTCUSDT-15m-2017-07.zip

File Download: /kaggle/working/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/15m/BTCUSDT-15m-2017-08.zip
[##################################################]
File Download: /kaggle

100%|██████████| 92/92 [00:03<00:00, 24.19it/s]
